In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir(os.path.join('..', 'notebook_format'))
from formats import load_style
load_style(plot_style = False)

In [2]:
os.chdir(path)

# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. a ipython magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import time
import threading

%watermark -a 'Ethen' -d -t -v

Ethen 2017-08-28 10:00:08 

CPython 3.5.2
IPython 5.4.1


# Threading

Threading allows us to run multiple tasks concurrently, so if we have two tasks on our hand, task A and task B, we can run them simultaneously without having to wait for task A to finish before running running task B. Let's look at some examples of working with threads. Before we create/initialize a thread, we define a simple function that simply sleeps for a specified amount of time.

In [3]:
def sleeper(n_time, name):
    print('I am {}. Going to sleep for {} seconds'.format(name, n_time))
    time.sleep(n_time)
    print('{} has woken up from sleep'.format(name))

We then initialize our thread with the `Thread` class from the `threading` module.

- `target`: accepts the function that we're going to execute
- `name`: is basically just naming the thread; this allows us to easily differentiate between threads when we have multiple threads
- `args`: pass in the argument to our function here

In [4]:
# we call .start to start executing the function from the thread
n_time = 2
thread = threading.Thread(target = sleeper, name = 'thread1', args = (n_time, 'thread1'))
thread.start()

I am thread1. Going to sleep for 2 seconds
thread1 has woken up from sleep


When we run a program and something is sleeping for a few seconds, we would have wait for that portion to wake up before we can continue with the rest of the program, but the concurrency of threads can bypass this behavior. Suppose we consider the main program as the main thread and our thread as its own separate thread, the code chunk below demonstrates the concurrency property, i.e. we don't have to wait for the main calling thread, i.e. the Python interpreter thread to finish before running the rest of our program.

In [5]:
# hello is printed before the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread1', args = (n_time, 'thread2'))
thread.start()

print()
print('hello')

I am thread2. Going to sleep for 2 seconds

hello
thread2 has woken up from sleep


Sometimes, we don't want Python to switch to the main thread until the thread we defined has finished executing its function. To do this, we can use `.join` method, this is essentially what they called the blocking call. It blocks the interpreter from accessing or executing the main program until the thread finishes it task.

In [6]:
# hello is printed after the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread1', args = (n_time, 'thread2'))
thread.start()
thread.join()

print()
print('hello')

I am thread2. Going to sleep for 2 seconds
thread2 has woken up from sleep

hello


## Multithreading

The following code chunk showcase how to initialize and utilize multiple threads.

In [7]:
n_time = 2
n_threads = 5
start = time.time()

# create n_threads number of threads and store them in a list
threads = []
for i in range(n_threads):
    name = 'thread_{}'.format(i)
    args = n_time, name
    thread = threading.Thread(target = sleeper, name = name, args = args)
    threads.append(thread)
    # we can start the thread while we're creating it, or move
    # this to its own loop
    thread.start()

# we could instead start the thread in a separate loop
# for thread in threads:
#     thread.start()

# ensure all threads have finished before executing main program
for thread in threads:
    thread.join()

elapse = time.time() - start
print()
print('Elapse time: ', elapse)

I am thread_0. Going to sleep for 2 seconds
I am thread_1. Going to sleep for 2 seconds
I am thread_2. Going to sleep for 2 seconds
I am thread_3. Going to sleep for 2 secondsI am thread_4. Going to sleep for 2 seconds

thread_0 has woken up from sleepthread_2 has woken up from sleepthread_3 has woken up from sleepthread_1 has woken up from sleepthread_4 has woken up from sleep




Elapse time: 
 2.0075490474700928


From the result above, we can observe that the elapse time that it doesn't take n_threads * (the time we told the sleep function to sleep) amount of time to finish all the task. Next we'll take a look at places where threads in Python would disappoint us. 

The following code chunk simply generate a random number and append that number to a list and we would repeat this operation for a specified number of times. The expectation is that on a multi-core machine a multithreaded code should make use of these extra cores and thus increase overall performance.

In [8]:
import random


def list_append(out_list, count):
    """
    Creates an empty list and then appends a 
    random number to the list 'count' number
    of times. A CPU-heavy operation!
    """
    for i in range(count):
        out_list.append(random.random())

In [9]:
count = 50000
start = time.time()
for i in range(n_threads):
    out_list = []
    list_append(out_list, count)

elapse = time.time() - start
print('Elapse time: ', elapse)

Elapse time:  0.040194034576416016


In [10]:
start = time.time()
threads = []
n_threads = 5
for i in range(n_threads):
    name = 'thread_{}'.format(i)
    out_list = []
    thread = threading.Thread(target = list_append, name = name, args = (out_list, count))
    threads.append(thread)
    thread.start() 

for thread in threads:
    thread.join()

elapse = time.time() - start
print('Elapse time: ', elapse)

Elapse time:  0.04251909255981445


With the 5 threads that we've specified, we might expect less than 5X speedup (less than 5 because we expect there's always going to overhead associated with creating the threads and managing the coordination between them). But in this example, the performance is even slightly worse than the serial implementation. So why is this the case?

This is due to the **GIL (Global Interpreter Lock)** in Python that prevents threads from actually running in parallel. The GIL is necessary because the Python interpreter is not thread safe. This means that there is a globally enforced lock when trying to safely access Python objects from within threads. At any one time only a single thread can acquire a lock for a Python object or C API. This is the reason that makes threads unsuitable for CPU intensive tasks in Python.

On the other hand, threads are very efficient and beneficial for task that are not CPU intensive. The benefit of threading in Python appears when our problems are network bound or data input/output (I/O) bound. This means that the Python interpreter is waiting for the result of a function call that's manipulating with data from an external source, such as network address or hard disk. 

For example, consider a Python code that is scraping many web URLs. By adding a new thread for each download resource, the code can download multiple data sources in parallel and combine the results at the end of every download. This means that each subsequent download is not waiting on the download of earlier web pages. In this case the program is now bound by the bandwidth limitations of the client/server(s) instead.

## Locks

The next topic is to introduce **Locks**. Locks are used when multiple threads are trying to access the same variable. By using locks we can guard ourselves from accessing the same object from multiple threads simultaneously, which can potentially corrupt our data.

For example, consider we have a program that does some kind of I/O processing and simply keeps track of how many items have we processed.

In [11]:
class Counter:
    """Just count some stuff"""
    def __init__(self, count = 0):
        self.count = count
        
    def increment(self):
        self.count += 1


def worker(counter, n_times):
    """increment by Counter class object for n_times"""
    for _ in range(n_times):
        counter.increment()


def run_threads(n_threads, target, args):
    threads = []
    for i in range(n_threads):
        thread = threading.Thread(target = target, args = args)
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

In [12]:
n_threads = 5
n_times = 10 ** 5
counter = Counter()
args = counter, n_times

run_threads(n_threads, target = worker, args = args)
print('count should be {}, got {}'.format(n_threads * n_times, counter.count))

count should be 500000, got 400154


Notice that when we ran the program from more than one thread, we find that the final counter value isn't necessarily correct (you can get correct results from time to time, but keep in mind that that's just a coincidence). The reason for this is that the increment operation is actually executed in three steps behind the scenes.

1. The Python interpreter fetches the current value of the counter
2. Calculates the new value (in this case just increment it)
3. Finally, writes the new value back to the variable

```python
# normal increment for thread A
value_a = counter.count
value_a += 1
counter.count = value_a
```

This can be problematic if another thread gets a hold of the old variable before the current thread writes back the new value. i.e.

```python
# thread A
value_a = counter.count
# thread B
value_b = counter.count

# thread A and thread B both increment on the old count
value_b += 1
value_a += 1
counter.count = value_b
counter.count = value_a
```

Since both threads are seeing the same original value, only one increment will be accounted for. To prevent this type of problems from occuring when accessing a shared resource, the threading module provides the Lock class. By using the lock, only one thread will be able to acquire the lock at any given point of time. If a thread attemps to hold a lock that's already held by some other thread, its exeuction is postponed until the lock is released.

In [13]:
class Counter:
    
    def __init__(self, count = 0):
        self.count = count
        self._lock = threading.Lock()
        
    def increment(self):
        """
        putting the shared resource count
        within the with block will acquire
        and release the lock for us
        """
        with self._lock:
            self.count += 1

In [14]:
n_threads = 5
n_times = 10 ** 5
counter = Counter()
args = counter, n_times
run_threads(n_threads, target = worker, args = args)
print('count should be {}, got {}'.format(n_threads * n_times, counter.count))

count should be 500000, got 500000


## Queue

Queues are basically a container of data/commands that are waiting to be retrieved. We can utilize them to control the flow of our tasks. Say we're receiving data from a website, we need to manipulate the data and write the data to a file. We have this three step process where each step sort of relies on the previous step (we can't manipulate the data if we haven't received the data and we don't want to save the data to the file if it hasn't gone through the preprocessing step). Queues help ensure this workflow is done in a very clean and organized manner.

In [16]:
# FIFO (First in First out, whatever we put in first will be
# the first that gets pulled out)
# LIFO
# Priority
from queue import Queue

# initialize the queue
queue = Queue()

# put items in the queue
queue.put(5)

# get the item out of the queue
print(queue.get())

# the queue is now empty because
# we put 1 item in and pulled 1 item out
print(queue.empty())

5
True


In [18]:
queue = Queue()

def consumer(queue):
    print('Consumer waiting')
    queue.get()                # Runs after put() below
    print('Consumer done')

thread = threading.Thread(target = consumer, args = (queue,))
thread.start()

# Example 12
print('Producer putting')
queue.put(1)            # Runs before get() above
thread.join()
print('Producer done')

Consumer waiting
Producer puttingConsumer done

Producer done


In [ ]:
def putting(queue):
    while True:

When using queues one important thing to note is that if we try to pull more items than there are available from the queue our program will freeze.

## Daemon Threads

Daemon is an attribute for our threads. i.e. when we initialize the threads we have an option to set daemon as true or false. What daemon does is that it ends the thread when the main program finishes. 

## Subclassing

When we subclass the `Thread` class we actually gain some flexibility such as making tweaks as to how the threads execute. When subclassing the `Thread` class it's advisable to only make changes to the `__init__` and `run` method.

The `run` method is closely related to the `.start()` method that we're calling. What happens is when we call the `.start()` method it actually calls the `run` method underneath the hood. The `run` method is then responsible for running the function that we gave. Let's look at the source code of the `run` method (with some additional added comments) to see where we can make our tweaks.

```python
def run(self):
    """Method representing the thread's activity.
    You may override this method in a subclass. 
    The standard run() method
    invokes the callable object passed to 
    the object's constructor as the
    target argument, if any, with sequential and 
    keyword arguments taken
    from the args and kwargs arguments, respectively.
    """
    try:
        # target is the callable object, a fancy word for
        # function, that we gave to the Thread class;
        # and the *self._args and **self._kwargs is simply
        # unpacking the argument or keyword argument that we gave
        # to the function
        if self._target:
            self._target(*self._args, **self._kwargs)
    finally:
        # Avoid a refcycle if the thread is running a function with
        # an argument that has a member that points to the thread.
        del self._target, self._args, self._kwargs
```

In [ ]:
thread = threading.Thread(target = sleeper, name = 'thread1', args = (3, 'thread1'))
thread.start()

So to actually run tasks concurrently, Python uses another technique called task switching. Python rapidly switches between each thread to make it seem like our program is actually running multiple tasks in parallel.

This can actually be very useful for event-driven tasks that have a lot of downtime and are waiting for the user to input something.

Threads are really efficient at using any downtime or idle time, so when one thread is waiting for something we can have another thread performing another task. So in our first example, when one thread is sleeping or not using much of the CPU resources, we can switch to execute another task to make sure the CPU is always busy.

When we have something that's CPU intensive, there's actually something called multiprocessing.

# Threads vs Processes

A process is an instance of program (e.g. Jupyter notebook, Google Chrome, Python interpreter). Processes spawn threads (sub-processes) to handle subtasks like reading keystrokes, loading HTML pages, saving files. Threads live inside processes and share the same memory space (they can read and write to the same variables).

Ex: Microsoft Word

When we open Word, we create a process (an instance of the program). When we start typing, the process spawns a number of threads: one to read keystrokes, another to display text on the screen, a thread to autosave our file, and yet another to highlight spelling mistakes. By spawning multiple threads, Microsoft takes advantage of "wasted CPU time" (waiting for our keystrokes or waiting for a file to save) to provide a smoother user interface and make us more productive.

# Multiprocessing

# Reference

- [Blog: Thread Synchronization Mechanisms in Python](http://effbot.org/zone/thread-synchronization.htm)
- [Forum: Python Parallel Processing - Tips and Applications](http://forums.fast.ai/t/python-parallel-processing-tips-and-applications/2092)
- [Youtube: Python Threading - Multithreading Playlist](https://www.youtube.com/playlist?list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm)